# Speed Up IGamma

In [1]:
# imports
from importlib import reload

import numpy as np
from scipy.special import gammainc
from scipy import interpolate

import mpmath
from zdm import energetics

# Time simple callscipy.special

In [2]:
gamma = -1.1
Emin = 1e31
Emax = 1e42

In [3]:
%timeit -r 10 norm = float(mpmath.gammainc(gamma, a=Emin/Emax))

324 µs ± 14.2 µs per loop (mean ± std. dev. of 10 runs, 1000 loops each)


# Time the array loop

In [4]:
Eth = 10**np.linspace(41., 43., 100)
Eth_Emax = Eth/Emax

In [5]:
# If this is too slow, we can adopt scipy + recurrance
%timeit -r 10 numer = np.array([float(mpmath.gammainc(gamma, a=iEE)) for iEE in Eth_Emax])

42.8 ms ± 1.41 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


# Real example

In [6]:
params = (1000000000000000000000000000000, 2.5118864315095718e+41, -1.01)

In [7]:
Emin = params[0]
Emax = params[1]
gamma = params[2]

In [8]:
Eth = np.load('Eth.npy')

In [9]:
Eth.shape

(500, 1400)

In [10]:
reload(energetics)
%timeit -r 2  result = energetics.array_cum_gamma(Eth, params[0], params[1], params[2])

2min 20s ± 2.51 s per loop (mean ± std. dev. of 2 runs, 1 loop each)


----

# Spline

## Explore

In [14]:
Eth.min()/Emax, Eth.max()/Emax

(0.005861653895076074, 2787.3410080677972)

## Init

In [15]:
avals = 10**np.linspace(-6, 6., 1000)

In [17]:
numer = np.array([float(mpmath.gammainc(
        gamma, a=iEE)) for iEE in avals])

In [18]:
tck = interpolate.splrep(avals, numer)

## Try it

In [21]:
def vector_cum_gamma_spline(tck, Eth, *params):
    params=np.array(params)
    Emin=params[0]
    Emax=params[1]
    gamma=params[2]

    # Calculate
    norm = Emax*float(mpmath.gammainc(gamma, a=Emin/Emax))
    Eth_Emax = Eth/Emax
    # If this is too slow, we can adopt scipy + recurrance
    numer = interpolate.splev(Eth_Emax, tck)
    result=numer/norm

    # Low end
    low= Eth < Emin
    result[low]=1.
    #high=np.where(Eth > Emax)[0]
    #if len(high)>0:
    #    result[high]=0.
    return result

In [22]:
def array_cum_gamma_spline(tck, Eth, *params):
    dims=Eth.shape
    result=vector_cum_gamma_spline(tck, Eth.flatten(),*params)
    result=result.reshape(dims)
    return result

In [25]:
%timeit -r 10 result2 = array_cum_gamma_spline(tck, Eth, params[0], params[1], params[2])
#np.array([float(mpmath.gammainc(gamma, a=iEE)) for iEE in Eth_Emax])

83 ms ± 5.19 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [24]:
(2*60+37)*1000 / 85.

1847.0588235294117

## Check accuracy

In [32]:
result2 = array_cum_gamma_spline(tck, Eth, params[0], params[1], params[2])
result = energetics.array_cum_gamma(Eth, params[0], params[1], params[2])

In [33]:
np.median(result-result2)

7.961669330157126e-167

In [34]:
np.std(result-result2)

7.229704916836677e-61

----

# Vectorize input?

----

# Check scipy

----

# Boost

### https://www.boost.org/doc/libs/1_78_0/libs/math/doc/html/math_toolkit/sf_gamma/igamma.html 

# C examples

### https://people.sc.fsu.edu/~jburkardt/c_src/asa239/asa239.html 

# More

### https://scicomp.stackexchange.com/questions/3341/fast-and-accurate-double-precision-implementation-of-incomplete-gamma-function 

# PyGSL

### https://github.com/pygsl/pygsl

----

# Checking mpmath

## Python

In [20]:
mpmath.gammainc(gamma, a=1)

mpf('0.1479904845936528')

In [14]:
gamma

-1.01

### Regularized

In [17]:
float(mpmath.gammainc(gamma, a=1, regularized=True))

0.0014859782602998275

## $\Gamma(-1.01)$

In [21]:
mpmath.gamma(gamma)

mpf('99.591285113277905')

## More

In [19]:
mpmath.gammainc(2+3j, 4, 10)

mpc(real='0.0097721266862770516', imag='-0.077063730631298996')

## Wolfram


### Matches on $\Gamma(-1.01)$

### Matches on the non-regularized evaluation too